In [ ]:
# CÉLULA DE CARGA - EXECUTAR UMA VEZ NO TOPO DO NOTEBOOK
import pandas as pd
import numpy as np

# Carrega os dados brutos
df_full_raw = pd.read_csv('../data/raw/campeonato-brasileiro-full.csv')
print("Dados brutos carregados com sucesso!")

Dados brutos carregados com sucesso!


In [2]:
# Copia dos arquivos brutos
df = df_full_raw.copy()
print("Cópia de trabalho 'df' criada.")


Cópia de trabalho 'df' criada.


In [3]:
# 2. Limpeza e Filtro Inicial
# Converter a coluna 'Data' para o formato datetime
df['data'] = pd.to_datetime(df['data'], dayfirst=True)

# Filtro para manter apenas jogos de 2014 em diante
df = df[df['data'].dt.year >= 2014].copy()

# Ordenar os jogos cronologicamente
df.sort_values(by='data', inplace=True)

In [4]:
# 3. Criar a Variável target
# 0 = Vitória do Mandante, 1 = Empate, 2 = Vitória do Visitante
def criar_target(row):
    if row['vencedor'] == row['mandante']:
        return 0
    elif row['vencedor'] == '-':
        return 1
    else:
        return 2

df['target'] = df.apply(criar_target, axis=1)

print("Limpeza, filtro e criação do Alvo concluídos.")
print(df[['data', 'mandante', 'visitante', 'vencedor', 'target']].head())

Limpeza, filtro e criação do Alvo concluídos.
           data       mandante    visitante       vencedor  target
4606 2014-04-19     Fluminense  Figueirense     Fluminense       0
4607 2014-04-19  Internacional      Vitoria  Internacional       0
4608 2014-04-19    Chapecoense     Coritiba              -       1
4609 2014-04-20          Bahia     Cruzeiro       Cruzeiro       2
4610 2014-04-20      Sao Paulo  Botafogo-RJ      Sao Paulo       0


In [ ]:
print(df.columns)

In [5]:
# PADRONIZAÇÃO DE NOMES DAS COLUNAS

df.columns = df.columns.str.lower()

# df.rename(columns={'mandante_Placar': 'mandante_placar'}, inplace=True)

print("Nomes de colunas padronizados para snake_case:")
print(df.columns)

Nomes de colunas padronizados para snake_case:
Index(['id', 'rodata', 'data', 'hora', 'mandante', 'visitante',
       'formacao_mandante', 'formacao_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'vencedor', 'arena', 'mandante_placar',
       'visitante_placar', 'mandante_estado', 'visitante_estado', 'target'],
      dtype='object')


In [6]:
# 4. Engenharia de "Forma"
df['pontos_mandante'] = np.where(df['target'] == 0, 3, np.where(df['target'] == 1, 1, 0))
df['pontos_visitante'] = np.where(df['target'] == 2, 3, np.where(df['target'] == 1, 1, 0))

df_unpivot = df.reset_index()

# Reestruturar para visão por time (unpivot), usando os nomes padronizados
df_team_stats = pd.concat([
    df_unpivot.rename(columns={'mandante': 'time', 'visitante': 'oponente', 'mandante_placar': 'gols_feitos', 'visitante_placar': 'gols_sofridos', 'pontos_mandante': 'pontos'}),
    df_unpivot.rename(columns={'visitante': 'time', 'mandante': 'oponente', 'visitante_placar': 'gols_feitos', 'mandante_placar': 'gols_sofridos', 'pontos_visitante': 'pontos'})
]).sort_values(by=['time', 'data'])

# Calcular as médias móveis
stats_to_roll = ['gols_feitos', 'gols_sofridos', 'pontos']
rolling_stats = df_team_stats.groupby('time')[stats_to_roll].rolling(window=5, min_periods=1).mean().shift(1)
rolling_stats.rename(columns=lambda x: f'form_{x}', inplace=True)

# Juntar as estatísticas de forma de volta
df_team_stats.reset_index(drop=True, inplace=True)
df_form = pd.concat([df_team_stats[['index', 'time']], rolling_stats.reset_index(drop=True)], axis=1)

# merge
df_final = df_unpivot.merge(df_form, left_on=['index', 'mandante'], right_on=['index', 'time'], suffixes=('', '_mandante'))
df_final = df_final.merge(df_form, left_on=['index', 'visitante'], right_on=['index', 'time'], suffixes=('_mandante', '_visitante'))

# Remover colunas duplicadas ou desnecessárias após os merges
df_final.drop(columns=['time_mandante', 'time_visitante'], inplace=True)

print("Engenharia de 'Forma' concluída.")
display(df_final[['mandante', 'visitante', 'form_pontos_mandante', 'form_pontos_visitante']].tail())

Engenharia de 'Forma' concluída.


,mandante,visitante,form_pontos_mandante,form_pontos_visitante
4174,Flamengo,Vitoria,2.2,1.6
4175,Bahia,Atletico-GO,0.8,1.0
4176,Juventude,Cruzeiro,2.2,1.0
4177,Atletico-MG,Athletico-PR,0.4,1.6
4178,Gremio,Corinthians,1.2,3.0


In [7]:
# 5. Engenharia de "Contexto" (Tática)

# Feature 1: Clássico Estadual
df_final['eh_classico'] = (df_final['mandante_estado'] == df_final['visitante_estado']).astype(int)

# Feature 2: Extrair números das formações táticas
def extrair_partes_formacao(formacao):
    # Função que lida com dados faltantes (NaN) ou formatos inesperados
    if pd.isna(formacao) or '-' not in str(formacao):
        return [3, 4, 3] # Retorna uma formação padrão/neutra se o dado for inválido
    
    parts = str(formacao).split('-')
    try:
        if len(parts) == 3: # Ex: 4-4-2
            return [int(p) for p in parts]
        elif len(parts) == 4: # Ex: 4-2-3-1
            # Soma os meio-campistas: (2+3=5) -> [4, 5, 1]
            return [int(parts[0]), sum(int(p) for p in parts[1:-1]), int(parts[-1])]
        else:
            return [3, 4, 3] # Retorno seguro para outros formatos
    except (ValueError, TypeError):
        return [3, 4, 3] # Retorno seguro se a conversão falhar

# Aplicar a função para mandante e visitante
formacoes_mandante = df_final['formacao_mandante'].apply(extrair_partes_formacao)
formacoes_visitante = df_final['formacao_visitante'].apply(extrair_partes_formacao)

df_final[['mandante_def', 'mandante_mid', 'mandante_att']] = pd.DataFrame(formacoes_mandante.tolist(), index=df_final.index)
df_final[['visitante_def', 'visitante_mid', 'visitante_att']] = pd.DataFrame(formacoes_visitante.tolist(), index=df_final.index)

# Feature 3: Criar features de diferença tática
df_final['diff_def'] = df_final['mandante_def'] - df_final['visitante_def']
df_final['diff_mid'] = df_final['mandante_mid'] - df_final['visitante_mid']
df_final['diff_att'] = df_final['mandante_att'] - df_final['visitante_att']

print("Engenharia de 'Contexto' concluída.")
display(df_final[['mandante', 'visitante', 'eh_classico', 'formacao_mandante', 'mandante_def', 'diff_att']].tail())

Engenharia de 'Contexto' concluída.


,mandante,visitante,eh_classico,formacao_mandante,mandante_def,diff_att
4174,Flamengo,Vitoria,0,4-4-2,4,-1
4175,Bahia,Atletico-GO,0,4-2-3-1,4,0
4176,Juventude,Cruzeiro,0,4-2-3-1,4,0
4177,Atletico-MG,Athletico-PR,0,4-3-3,4,2
4178,Gremio,Corinthians,0,4-4-2,4,0


In [8]:
# 6. Preparar DataFrame Final para o Modelo

# Lista de todas as features que criamos, usando os nomes padronizados
feature_cols = [
    'form_gols_feitos_mandante', 'form_gols_sofridos_mandante', 'form_pontos_mandante',
    'form_gols_feitos_visitante', 'form_gols_sofridos_visitante', 'form_pontos_visitante',
    'eh_classico',
    'mandante_def', 'mandante_mid', 'mandante_att',
    'visitante_def', 'visitante_mid', 'visitante_att',
    'diff_def', 'diff_mid', 'diff_att'
]

target_col = 'target'

# Selecionar apenas as colunas úteis e criar uma cópia para evitar avisos
df_model = df_final[feature_cols + [target_col]].copy()

# Remover quaisquer linhas que ainda tenham valores nulos 
# (gerados pelo .shift() nas primeiras partidas de cada time no dataset)
df_model.dropna(inplace=True)

print("DataFrame final 'df_model' pronto para treinamento!")
print("Dimensões:", df_model.shape)
display(df_model.head())

DataFrame final 'df_model' pronto para treinamento!
Dimensões: (4178, 17)


,form_gols_feitos_mandante,form_gols_sofridos_mandante,form_pontos_mandante,form_gols_feitos_visitante,form_gols_sofridos_visitante,form_pontos_visitante,eh_classico,mandante_def,mandante_mid,mandante_att,visitante_def,visitante_mid,visitante_att,diff_def,diff_mid,diff_att,target
0,2.0,1.0,2.2,0.8,1.6,0.2,0,3,4,3,3,4,3,0,0,0,0
1,1.2,1.6,1.2,1.4,1.4,1.4,0,3,4,3,3,4,3,0,0,0,0
2,1.0,1.4,0.6,3.0,0.8,3.0,0,3,4,3,3,4,3,0,0,0,1
3,1.2,1.0,1.4,0.6,2.6,0.2,0,3,4,3,3,4,3,0,0,0,2
4,0.4,1.8,0.4,1.2,1.4,1.4,0,3,4,3,3,4,3,0,0,0,0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib

# 1. Separar as Features (X) do Alvo (y)
X = df_model.drop('target', axis=1)
y = df_model['target']

# 2. Dividir os dados em Treino e Teste
# Usaremos 80% para treinar o modelo e 20% para testá-lo em dados que ele nunca viu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensões dos dados de treino:", X_train.shape)
print("Dimensões dos dados de teste:", X_test.shape)

# 3. Treinar e Avaliar o Modelo Baseline (Regressão Logística)
print("\n--- Treinando Modelo Baseline ---")
log_reg = LogisticRegression(max_iter=1000) 
log_reg.fit(X_train, y_train)
predictions_log_reg = log_reg.predict(X_test)
accuracy_log_reg = accuracy_score(y_test, predictions_log_reg)
print(f"Acurácia da Regressão Logística: {accuracy_log_reg:.4f}")

# 4. Treinar e Avaliar o Modelo Avançado (XGBoost)
print("\n--- Treinando Modelo Avançado ---")
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)
predictions_xgb = xgb_model.predict(X_test)
accuracy_xgb = accuracy_score(y_test, predictions_xgb)
print(f"Acurácia do XGBoost: {accuracy_xgb:.4f}")

# 5. Salvar o melhor modelo
# O XGBoost provavelmente terá uma performance superior, então vamos salvá-lo
model_filename = '../models/fut_br_predictor_model.joblib'
joblib.dump(xgb_model, model_filename)
print(f"\nModelo XGBoost salvo com sucesso em: {model_filename}")

Dimensões dos dados de treino: (3342, 16)
Dimensões dos dados de teste: (836, 16)

--- Treinando Modelo Baseline ---
Acurácia da Regressão Logística: 0.5024

--- Treinando Modelo Avançado ---
Acurácia do XGBoost: 0.4569

Modelo XGBoost salvo com sucesso em: ../models/fut_br_predictor_model.joblib


c:\Users\berna\OneDrive\Área de Trabalho\data_eng_campeonato_brasileiro\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:52:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [13]:
# CÉLULA PARA GERAR DADOS DE TESTE

import json

# Pega a primeira linha do nosso conjunto de teste
dados_para_teste = X_test.iloc[0]

# Converte a Series do pandas para um dicionário Python padrão
dados_dicionario = dados_para_teste.to_dict()


# Converte o dicionário Python para uma string no formato JSON VÁLIDO
# O indent=4 formata o texto de forma legível
dados_json_string = json.dumps(dados_dicionario, indent=4)

print("Copie e cole o texto JSON abaixo no 'Request body' da API:")
print(dados_json_string)

Copie e cole o texto JSON abaixo no 'Request body' da API:
{
    "form_gols_feitos_mandante": 1.2,
    "form_gols_sofridos_mandante": 1.4,
    "form_pontos_mandante": 1.4,
    "form_gols_feitos_visitante": 1.6,
    "form_gols_sofridos_visitante": 1.4,
    "form_pontos_visitante": 1.6,
    "eh_classico": 0.0,
    "mandante_def": 4.0,
    "mandante_mid": 5.0,
    "mandante_att": 1.0,
    "visitante_def": 4.0,
    "visitante_mid": 5.0,
    "visitante_att": 1.0,
    "diff_def": 0.0,
    "diff_mid": 0.0,
    "diff_att": 0.0
}
